In [2]:
import pandas
import numpy
import matplotlib
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import os
import cv2
import random

In [3]:
labels = []
all2test = []
count = 0
for filename in os.listdir("images"):
    path = os.path.join("images", filename)
    img = cv2.imread(path)
    if count < 18000:
        labels.append(0)
        count += 1
    elif count < 24000:
        labels.append(0)
        count += 1
    elif count < 30000:
        labels.append(1)
        count += 1
    else:
        labels.append(1)
    all2test.append(img)